In [2]:
import bokeh
from bokeh.plotting import figure, show, output_notebook
from bokeh.charts import Scatter, Line, color
from bokeh import palettes
from bokeh.layouts import row
import colorcet
output_notebook()

import numpy as np
import pandas as pd
import os
import math
import time
from IPython.display import clear_output

colors = palettes.Paired12
basedirs = ['results',
            '~/vlg4/structure-experiments/results']
# basedirs = ['/mnt/nyu/structure-experiments/results']

tooltips = [
    ('Experiment', '@experiment'),
#     ('Step', '@step'),
#     ('Loss', '@loss'),
#     ('Prior divergence', '@priordivergence'),
#     ('Transition divergence', '@transdivergence'),
]

Loading BokehJS ...

In [3]:
def sparsify(steps, values, amount):
    if amount < 2: return steps, values
    sparse_steps = []
    sparse_values = []

    for i in range(min(len(steps), len(values))):
        if i % amount == 0:
            sparse_values.append(values[i])
            sparse_steps.append(steps[i])
    return np.array(sparse_steps), np.array(sparse_values)

In [4]:
def load_result_df(experiment, smoothing=1, max_points=500, upto=None, jitter=0):
    for basedir in basedirs:
        df = pd.DataFrame()
        try:
            path = os.path.join(basedir, experiment, 'results.csv')
            df = pd.read_csv(path, error_bad_lines=False)
            if len(df) > 0:
                break
        except Exception as e:
            import traceback
#             traceback.print_exc()
#             print("Error in parsing experiment {} from basedir {}:".format(experiment, basedir), e)
            continue

    numeric_columns = list(df.columns)
    df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors="coerce")

    if upto is not None:
        df = pd.DataFrame(df.query("step<{}".format(upto)))
    
    for col in numeric_columns:
        df[col] = df[col].rolling(smoothing, center=True).mean()
        if jitter != 0 and col != 'step':
            df[col] = df[col] + np.random.randn(len(df[col])) * jitter
    
    for col in df:
        df["".join(col.split()).lower()] = df[col]
    df['experiment'] = pd.Series([experiment] * len(df), index=df.index)
    
    every_nth = max(int(len(df) / max_points), 1)
    return df.iloc[::every_nth, :]

def make_dataframe(results, **kwargs):
    data = pd.DataFrame()
    for experiment in results:
        single_df = load_result_df(experiment, **kwargs)
        data = data.append(single_df, ignore_index=True)
    return data

In [5]:
title = "colored moving mnist"
networks = [
    "adversarial-0.01",
    "adversarial-0.001",
    "adversarial-0.0001",
    "adversarial-0.01-1e-4",

]

data = make_dataframe(networks, smoothing=1, max_points=1000, upto=2e7)

spaced_palette = palettes.linear_palette(colorcet.rainbow, len(networks))

plots = []
for facet in ['autoencoder_loss', 'reconstruction_loss', 'adversarial_loss', 'adversary_loss', 'true_outputs', 'false_outputs']:
    plot = Line(data, x='step', y=facet,
                title=title + ", " + facet,
                color=color(columns='experiment', palette=spaced_palette),
                tools='pan,wheel_zoom,box_zoom,save,reset', 
                active_scroll="wheel_zoom",
                tooltips=tooltips,
                plot_height=300,
                legend='top_right',)
    plots.append(plot)
    show(row(children=[plot], responsive=True))